# 4. Exploratory Data Analaysis
---

### Introduction

***1. Main objective of EDA.***

The main objective of this Exploratory Data Analysis is to obtain information that can led the ML team to answers in order to predict a possible outcome of future cs-go matches. The hypotesis of the ML team is the following: ***It is possible to predict a cs-go match outcome by the information about the players.***

The dataset used for this EDA is player_match_data.csv, which is a combination from the originals match_players.csv and match_results.csv. and was done through SQL queries as part of the proyect requirements. The resultant dataset provides information of every match from 2012 to 2017 from a player's perspective. 

***2. Structure of the EDA.***
 
The Eda will be divide in two parts.In the **The first part**, we will use the given information to study the correlation between variables. It includes the data cleaning, target creation, multivariable analysis and outlier analysis. **The second part** will consist in shaping the dataset in order to fit the requirements of the Machine Learning Model. It will be developed in the feature engineering section.

***3. Issues of the dataset.***

Before starting the analysis, there are two main issues that must be corrected in order to continue. 
1. **There is no target variable**. Each row provides information about the player's performance in a match but there is no information about the outcome of the player.
2. **The player is not directly connected to a team**. Each row has the player id and the team name. Moreover, it has also the teams ids and scores. However, it falied to connect the team name provided with any of the team ids, so with the given information it is impossible to know if the player has won or lost the match. 

In order to solve this problems and continue with the EDA analysis, the following variables will be created:

1. **team_id**. This will be a id-variable that will bring the team id using the team name.

2. **winning team**. A categorical variable called 'winning_team' and consists in a number (0, 1 or 2). This variable will calculate the diference between team_1_score and team_2_score and will return the winner of the match.
    - 0 means draw,
    - 1 means that team 1 has won the match,
    - 2 means that team 2 has won the match.

    This variable will be analyized as a possible target variable. 

3. **winning_team_id**. This will be a id-variable that simply returns the winning team id. It uses the winning team outcome and returns the ID of one of both teams, or 0 if the match resulted in a draw. This variable will be only used to support the target variable.

4. **player has won**. A categorical target variable that provides 1 or 0 depending whether the player was part of the winning team or not. This will be calculated using the player_team_id and winning_team_id. If they are the same, it will return a 1. This means that both defeats and draws will be counted as 0.

It is important to state that once the variables have fullfiled their purpose - that is to develop de target variable successfully- they will be deleted.


***4. Feature engineering.***

The main objective of this project is to predict match outcomes based on players’ names. After analyzing and filtering the predictors by their correlation, several new features will be generated. This is necessary to reshape the information and enable the future machine learning model to analyze the relevant data. Further details will be explained in the corresponding section. 


### 4.0. Importing libraries & dataset and creating variables

***Importing Libraries and dataset***

In [593]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Dataset
total_data = pd.read_csv('../data/interim/player_match_data.csv')

***Overview of the dataset***

In [594]:
total_data.head(3)

,adr,assists,deaths,fkdiff,hs,kdratio,kills,rating,match_id,player_id,...,team_2_score,team_name,data_unix,map,hour,day,week,month,year,weekday
0,163.2,3,10,1,10,90.0%,32,2.44,32227,5736,...,16,g3x,1467476700000,Train,13,2,26,7,2016,5
1,81.0,3,6,1,5,75.0%,17,1.55,32227,2532,...,16,g3x,1467476700000,Train,13,2,26,7,2016,5
2,77.6,3,10,1,11,75.0%,16,1.41,32227,7382,...,16,g3x,1467476700000,Train,13,2,26,7,2016,5


***Creating variables***

In [595]:
    # Variable 1: player_team_id 

# 'team_id' function
def find_most_common_team_number(group):
        all_team_ids = pd.concat([group['team_1_id'], group['team_2_id']])
        team_numbers = all_team_ids.dropna().astype(int)
        most_common_team_number = np.argmax(np.bincount(team_numbers))
        return most_common_team_number

# Applying function to dataset
total_data['team_id'] = total_data.groupby('team_name').apply(find_most_common_team_number).reindex(total_data['team_name']).values

# ----------------------------------------------------------------------------------------------------------------------------

    # Variable 2: winning_team

# 'winning_team' function
def get_winning_team(team_1_score, team_2_score):
        if team_1_score == team_2_score: return 0
        elif team_1_score > team_2_score: return 1
        else: return 2

# Aapplying function to dataset
total_data['winning_team'] = total_data.apply(lambda row: get_winning_team(row['team_1_score'], row['team_2_score']), axis=1)

# ----------------------------------------------------------------------------------------------------------------------------

    # Variable 3: winning_team_id 

# Creating column called 'winning_team_id' (doesn't need function)
total_data['winning_team_id'] = np.where(total_data['winning_team'] == 1, total_data['team_1_id'],
                                np.where(total_data['winning_team'] == 2, total_data['team_2_id'], 0))

# ----------------------------------------------------------------------------------------------------------------------------

    # Variable 4: player_has_won 

# Creating column called 'player_has_won' (doesn't need function)
total_data['player_has_won'] = np.where(total_data['winning_team_id'] == total_data['team_id'], 1, 0)

# ----------------------------------------------------------------------------------------------------------------------------

# Seleccting target variables

target_1 = 'winning_team'
target_2 = 'player_has_won'

# Show small overview of the dataset
total_data.head(3)


,adr,assists,deaths,fkdiff,hs,kdratio,kills,rating,match_id,player_id,...,hour,day,week,month,year,weekday,team_id,winning_team,winning_team_id,player_has_won
0,163.2,3,10,1,10,90.0%,32,2.44,32227,5736,...,13,2,26,7,2016,5,6621,2,6621,1
1,81.0,3,6,1,5,75.0%,17,1.55,32227,2532,...,13,2,26,7,2016,5,6621,2,6621,1
2,77.6,3,10,1,11,75.0%,16,1.41,32227,7382,...,13,2,26,7,2016,5,6621,2,6621,1


After the four variables were created, it is possible to begin with the EDA analysis

---

### 4.1. Data description

***Dataset general information***

In [596]:
total_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946025 entries, 0 to 946024
Data columns (total 29 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   adr              946025 non-null  object 
 1   assists          946025 non-null  int64  
 2   deaths           946025 non-null  int64  
 3   fkdiff           946025 non-null  int64  
 4   hs               946025 non-null  int64  
 5   kdratio          946025 non-null  object 
 6   kills            946025 non-null  int64  
 7   rating           946025 non-null  float64
 8   match_id         946025 non-null  int64  
 9   player_id        946025 non-null  int64  
 10  team_1_id        946025 non-null  int64  
 11  team_1_name      946025 non-null  object 
 12  team_1_score     946025 non-null  int64  
 13  team_2_id        946025 non-null  int64  
 14  team_2_name      946025 non-null  object 
 15  team_2_score     946025 non-null  int64  
 16  team_name        946025 non-null  obje

    1. Dataset general information:

The dataset contains 946.025 rows and 27 columns (including the two possible target columns). It is possible to divide the columns in four different groups:

- **Group 1: Match statistics**, from columns 0 to 7. These includes all the statistics from the game, including one called 'rating', which was created by the owner of the dataset. 
- **Group 2: IDs and names** of team, match and players, from columns 8 to 14.
- **Group 3: Time data**, from 16 to 22. These includes all time features.
- **Group 4: New variables and target** from 23 to 26. 

#

    2. Data types:

It is possible to find three types of data in this dataset: Numerical variables (columns 0 to 7), factorized categorical variables (columns 17 to 22), categorical variables (column 16 - map), numerical ids and score (columns 8 to 14, 23 and 25) and the two target variables, which are categorical (0, 1). There is only a time series variable, column 15 - data_unix, that provide the exact time the match was played.

    3. Null values:

According to the general information, there are no null-values in the dataset. However, It is possible to see that three variables- adr, kdratio, map - are listed as object type and may contain invalid values in it. 


Due to this, the next step will be converting all the datatypes and inspecting if there are null variables in it.



### 4.2. Null values

***Converting 'adr' and 'kdratio' to numeric columns***

In [597]:
# Convert'adr' and 'kdratio' to numeric columns
total_data['adr'] = pd.to_numeric(total_data['adr'], errors='coerce')

# Kdratio is a % number, so it will be divided by 100
total_data['kdratio'] = pd.to_numeric(total_data['kdratio'].str.rstrip('%'), errors='coerce') / 100.0

# print a list with the info of the previous variables
total_data[['adr','kdratio']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946025 entries, 0 to 946024
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   adr      815862 non-null  float64
 1   kdratio  815988 non-null  float64
dtypes: float64(2)
memory usage: 14.4 MB


After the replacement and data type conversion its possible to see the following: 
- Column 'adr' has 130.163 null values, a 13.758% of the total entries.
- Column 'kdratio' has 130.037 null values, a 13.745% of the total entries.

***Analyze 'map' column***

In [598]:
total_data['map'].unique()

array(['Train', 'Dust2', 'Dust2_se', 'Cache', 'Mirage', 'Nuke', 'Inferno',
       'Cobblestone', 'Vertigo', 'Overpass', 'Nuke_se', 'Inferno_se',
       'Mirage_ce', 'Train_se', 'Season', 'Mill_ce', 'Tuscan', 'Ancient',
       'Dust_se', 'Train_ve'], dtype=object)

All the values in 'map' seems to be correct.

***Null-values replacement***

Since the variables 'adr' and 'kdratio' are numeric and both count with a normal distribution, the replacement of the null entries will follow this criteria:
1. Player's 'adr' or 'kdratio' will be filled with the player's mean of them.
2. If there is no information from the player to fill the nan, the row will be deleted.


In [599]:
# Group by 'player_id' and fill NaN values in each player's 'adr' column with the mean of non-null values for that player
total_data['adr'] = total_data.groupby('player_id')['adr'].transform(lambda x: x.fillna(x[x.notna()].mean()))
total_data['kdratio'] = total_data.groupby('player_id')['kdratio'].transform(lambda x: x.fillna(x[x.notna()].mean()))

# print a list with the info of the previous variables
total_data[['adr','kdratio']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 946025 entries, 0 to 946024
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   adr      930980 non-null  float64
 1   kdratio  931002 non-null  float64
dtypes: float64(2)
memory usage: 14.4 MB


After the replacement, 'adr' and 'kdratio' still have null values (15.045 and 15.023 respectively).
The rest of rows with null values (1.59% of the dataset) will be deleted.

In [600]:
# Saving a copy of the rows with null values
null_rows = total_data.loc[total_data.isnull().any(axis=1)]

# Deleting rows with null values
total_data = total_data.dropna()

# print a list with the info of the previous variables
total_data[['adr','kdratio']].info()

<class 'pandas.core.frame.DataFrame'>
Index: 930980 entries, 0 to 946024
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   adr      930980 non-null  float64
 1   kdratio  930980 non-null  float64
dtypes: float64(2)
memory usage: 21.3 MB


Without the null-values, the total length of the dataset is 930.980 rows.

### 4.3. Multivariable analysis

***Numerical - numerical analysis***

The following analysis will be held by parts. First, the *statistcs* group will be analyzed and then the *time* group. Both target will be taken into account.

In [601]:
"""numeric_variables = ['adr', 'assists', 'deaths', 'fkdiff', 'hs', 'kdratio', 'kills','rating']

for column in numeric_variables:
    # Setting up the subplots for target_1
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    # Regression plot for target_1
    sns.regplot(x=column, y=target_1, data=total_data, ax=axes[0])
    axes[0].set_title('Regplot: {} vs. {}'.format(column, target_1))

    # Heatmap of correlation for target_1
    sns.heatmap(total_data[[column, target_1]].corr().iloc[0:1, :], annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Correlation Heatmap: {} vs. {}'.format(column, target_1))

    # Show the plot for each column with target_1
    plt.tight_layout()
    plt.show()

    # Setting up the subplots for target_2
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    # Regression plot for target_2
    sns.regplot(x=column, y=target_2, data=total_data, ax=axes[0])
    axes[0].set_title('Regplot: {} vs. {}'.format(column, target_2))

    # Heatmap of correlation for target_2
    sns.heatmap(total_data[[column, target_2]].corr().iloc[0:1, :], annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Correlation Heatmap: {} vs. {}'.format(column, target_2))

    # Show the plot for each column with target_2
    plt.tight_layout()
    plt.show()"""


"numeric_variables = ['adr', 'assists', 'deaths', 'fkdiff', 'hs', 'kdratio', 'kills','rating']\n\nfor column in numeric_variables:\n    # Setting up the subplots for target_1\n    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))\n\n    # Regression plot for target_1\n    sns.regplot(x=column, y=target_1, data=total_data, ax=axes[0])\n    axes[0].set_title('Regplot: {} vs. {}'.format(column, target_1))\n\n    # Heatmap of correlation for target_1\n    sns.heatmap(total_data[[column, target_1]].corr().iloc[0:1, :], annot=True, cmap='coolwarm', ax=axes[1])\n    axes[1].set_title('Correlation Heatmap: {} vs. {}'.format(column, target_1))\n\n    # Show the plot for each column with target_1\n    plt.tight_layout()\n    plt.show()\n\n    # Setting up the subplots for target_2\n    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))\n\n    # Regression plot for target_2\n    sns.regplot(x=column, y=target_2, data=total_data, ax=axes[0])\n    axes[0].set_title('Regplot: {} v

After watching the numerical - numerical plots, it is possible to resume the information:

| Predictor | Corr with winning_team | Corr with player_has_won |
|-----------|-------------------------|---------------------------|
| adr       | -0.0025                 | 0.34                      |
| assists   | 0.005                   | 0.19                      |
| deaths    | 0.026                   | -0.41                     |
| fkdiff    | 0.00067                 | 0.27                      |
| hs        | 0.017                   | 0.19                      |
| kdratio   | 0.0074                  | 0.52                      |
| kills     | 0.019                   | 0.30                      |
| Rating    | 0.00072                 | 0.55                      |


It is important to state that the variable winning_team cannot be used as a target due to its low performance with other variables. Since 100% of the players have played as team 1 or team 2 during their career, this strongly affects the outcome of the variable. However, when the second path is taken and the variable player_has_won is analyized, the performance rise up to acceptable levels. This is a resume with the correlations: 

- **Rating** (0.55), **Kdratio** (0.52) and **deaths** (-0.41) have the strongest correlation.
- **adr** (0.34), **kills** (0.30) and **fkdiff** (0.27) has a good correlation.
- **assists** (0.19) and **hs** (0.19) have a poor correlation.

***Map factorization***

The only categorial variable that is has been not factorized is 'map'. The rest of the variables are already numbers. 

In [602]:
# Creating a column called 'map_n' that has the information factorized
total_data['map_n'], unique_maps = pd.factorize(total_data['map'])

***Categorical - numerical analysis***

In [603]:
"""

numeric_variables_2 = ['map_n', 'hour', 'day', 'week', 'month', 'year', 'weekday']

for column in numeric_variables_2:
    # Setting up the subplots for target_1
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    # Regression plot for target_1
    sns.regplot(x=column, y=target_1, data=total_data, ax=axes[0])
    axes[0].set_title('Regplot: {} vs. {}'.format(column, target_1))

    # Heatmap of correlation for target_1
    sns.heatmap(total_data[[column, target_1]].corr().iloc[0:1, :], annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Correlation Heatmap: {} vs. {}'.format(column, target_1))

    # Show the plot for each column with target_1
    plt.tight_layout()
    plt.show()

    # Setting up the subplots for target_2
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    # Regression plot for target_2
    sns.regplot(x=column, y=target_2, data=total_data, ax=axes[0])
    axes[0].set_title('Regplot: {} vs. {}'.format(column, target_2))

    # Heatmap of correlation for target_2
    sns.heatmap(total_data[[column, target_2]].corr().iloc[0:1, :], annot=True, cmap='coolwarm', ax=axes[1])
    axes[1].set_title('Correlation Heatmap: {} vs. {}'.format(column, target_2))

    # Show the plot for each column with target_2
    plt.tight_layout()
    plt.show()

    """

"\n\nnumeric_variables_2 = ['map_n', 'hour', 'day', 'week', 'month', 'year', 'weekday']\n\nfor column in numeric_variables_2:\n    # Setting up the subplots for target_1\n    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))\n\n    # Regression plot for target_1\n    sns.regplot(x=column, y=target_1, data=total_data, ax=axes[0])\n    axes[0].set_title('Regplot: {} vs. {}'.format(column, target_1))\n\n    # Heatmap of correlation for target_1\n    sns.heatmap(total_data[[column, target_1]].corr().iloc[0:1, :], annot=True, cmap='coolwarm', ax=axes[1])\n    axes[1].set_title('Correlation Heatmap: {} vs. {}'.format(column, target_1))\n\n    # Show the plot for each column with target_1\n    plt.tight_layout()\n    plt.show()\n\n    # Setting up the subplots for target_2\n    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))\n\n    # Regression plot for target_2\n    sns.regplot(x=column, y=target_2, data=total_data, ax=axes[0])\n    axes[0].set_title('Regplot: {} vs. {}

After the plotting, it is possible to resume the info in a table: 

| Variable | Corr. with Winning Team | Corr. with Player Has Won |
|----------|-------------------------|---------------------------|
| map      | -0.0029                 | -0.0031                   |
| hour     | -0.0026                 | -0.0032                   |
| day      | 0.0047                  | -0.00012                  |
| week     | -0.0054                 | 0.0024                    |
| month    | -0.0059                 | 0.0026                    |
| year     | 0.032                   | 0.0093                    |
| weekday  | -0.00096                | 0.0055                    |

In both cases, the correlations with the target variables are extremely low, meaning that they have almost no use in the analysis. In the overall, the first target (winning_team) performed better than the second target (player_has_won).

***General heatmap***

For the heatmap, onyl the statistical variables will be used since they are the ones with the best correlation with target.

In [604]:
"""

heatmap_data = total_data[['player_has_won', 
                 'adr', 
                 'assists', 
                 'deaths', 
                 'fkdiff', 
                 'hs', 
                 'kdratio', 
                 'kills',
                 'rating',]]

correlation_matrix = heatmap_data.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))

# Create a heatmap with Seaborn
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# Show the plot
plt.title('Correlation Heatmap')
plt.show()"""

'\n\nheatmap_data = total_data[[\'player_has_won\', \n                 \'adr\', \n                 \'assists\', \n                 \'deaths\', \n                 \'fkdiff\', \n                 \'hs\', \n                 \'kdratio\', \n                 \'kills\',\n                 \'rating\',]]\n\ncorrelation_matrix = heatmap_data.corr()\n\n# Set up the matplotlib figure\nplt.figure(figsize=(12, 10))\n\n# Create a heatmap with Seaborn\nsns.heatmap(correlation_matrix, annot=True, cmap=\'coolwarm\', fmt=".2f", linewidths=.5)\n\n# Show the plot\nplt.title(\'Correlation Heatmap\')\nplt.show()'

This graph shows that there are some strong correlations between variables in the dataset. For example, adr (average damage per round) and kills are also strongly correlated variables. It is possible to say that from the original dataset of 26 columns, the EDA found that only seven predictors are important to predict the possible outcome of a player in a CS-GO match.

There is a special mention of the variable **'rating'**. This is made up of other predictors, so its correlation with the rest is the highest in the dataset. However, we do not count with the information to know exactly how this variable is made. This could negatively affect the research and the ML model.

### 4.4 Outlier analysis

INCOMPLETE

### 4.5 Feature Engineering

The initial dataset was constructed from the player’s viewpoint. However, to align with the machine learning model’s requirements, we need to shift our focus to the team’s perspective. 

Consequently, we will reshape the dataset by merging and selectively deleting variables, emphasizing the team-centric information. This transformation will enable the model to accurately predict match outcomes.

***Deleting irrelevant variables***

- The group ID and name variables have been removed because it is not possible to use them for the machine learning model.
- Time group variables were deleted due to their poor performance with both target variables.
- Although the target 2 allowed us to discover the correlation between variables, the target needed for the ML model is the targe 1 -winning team-.

In [605]:
total_data.drop(['hour', 'day', 'week', 'month', 'weekday', 'year', 'rating', 'team_1_name', 'team_2_name', "map_n"], axis=1, inplace=True)
total_data.drop(['team_1_score', 'team_2_score', 'winning_team_id', 'player_has_won'], axis=1, inplace=True)
total_data.head(3)

,adr,assists,deaths,fkdiff,hs,kdratio,kills,match_id,player_id,team_1_id,team_2_id,team_name,data_unix,map,team_id,winning_team
0,163.2,3,10,1,10,0.90,32,32227,5736,6619,6621,g3x,1467476700000,Train,6621,2
1,81.0,3,6,1,5,0.75,17,32227,2532,6619,6621,g3x,1467476700000,Train,6621,2
2,77.6,3,10,1,11,0.75,16,32227,7382,6619,6621,g3x,1467476700000,Train,6621,2


***Create team_id variable***

The list used for merging with team id is from the team_data.csv

In [606]:
# #TAL VEZ ESTO NO SE USE
# #------------------------------------
# #Import .CSV file
# unique_teams = pd.read_csv('../data/interim/team_data.csv')

# # Group all team names to save memory
# unique_teams = unique_teams.groupby('team_name').first().reset_index()

# # Merge total_data with unique_teams based on 'team_name'
# total_data = pd.merge(total_data, unique_teams, on='team_name', how='left')

# # Fill NaN values in 'team_id' with -1 and convert to int
# total_data['team_id'] = total_data['team_id'].fillna(-1).astype(int)

# # Drop 'team name' variable as it will not be used anymore
# total_data = total_data.drop('team_name', axis=1)
# total_data = total_data.drop('Unnamed: 0', axis=1)

# total_data.head(20)

usando la formula de merge quedan 97870
usando la formula del propio dataset quedan 90440

***Create a team dataframe: Group all matchs by MatchID and TeamID***

A sub dataframe will be created with the team information of each match. This will provide a dataframe that has two rows for each match (1 per team). It will have all the players IDs, the mean of adr and kdratio and the sum of the rest of the variables. This will provide us the **teams' performance** in every match played. 

In [607]:
# Group by MatchID and TeamID, then calculate average stats and collect PlayerIDs
team_stats = total_data.groupby(['match_id', 'team_id']).agg({
    'player_id': lambda x: frozenset(x),
    'adr': 'mean',
    'assists': 'sum',
    'deaths': 'sum',
    'fkdiff': 'sum',
    'hs': 'sum',
    'kdratio': 'mean',
    'kills': 'sum',
}).reset_index()

team_stats.head(3)

,match_id,team_id,player_id,adr,assists,deaths,fkdiff,hs,kdratio,kills
0,12838,4411,"(884, 7148, 29, 39)",73.855910,0,38,6,0,0.694175,73
1,12838,4443,(7150),46.216667,0,19,-7,0,0.509667,5
2,12839,4411,"(884, 7148, 29, 39)",73.855910,0,30,11,0,0.694175,64


In [608]:
# Add a new column with the size of frozenset
team_stats['team_size'] = team_stats['player_id'].apply(len)

# Sort the DataFrame by the size of frozenset
team_stats = team_stats.sort_values(by='team_size')
match_ids = team_stats[team_stats['team_size'] != 5]['match_id'].to_list()
team_stats = team_stats[~team_stats['match_id'].isin(match_ids)]

# # Drop the temporary 'team_size' column if needed
team_stats.drop(columns=['team_size'], inplace=True)
team_stats.rename(columns={
    'player_id':'members',
    'adr': 'avg_adr',
    'assists': 'sum_assists',
    'deaths': 'sum_deaths',
    'fkdiff': 'sum_fkdiffs',
    'hs': 'sum_hs',
    'kdratio': 'mean_kdratio',
    'kills': 'sum_kills',
    }, inplace=True)

team_stats.head(3)

,match_id,team_id,members,avg_adr,sum_assists,sum_deaths,sum_fkdiffs,sum_hs,mean_kdratio,sum_kills
119692,82115,9085,"(13361, 9811, 12572, 9277, 9279)",72.10,15,87,-3,43,0.664,80
127058,86385,8848,"(12547, 10664, 10666, 10705, 14577)",72.34,23,86,0,38,0.700,93
119981,82269,9788,"(9806, 10992, 10993, 10994, 8891)",72.52,32,100,-5,33,0.669,91


***Merge the teams dataframe with the main dataframe***

The merging will provide the main dataset with the statistics related to the team. Now, every row will have the information on the player and the team views. However, the player information is not needed anymore, so everything but the players' id will be deleted. Finally, the dataframe will be grouped by match ID and team ID, giving two rows per match, one for each team.

In [609]:
# Merge with the original DataFrame to get PlayerIDs
merged_df = pd.merge(total_data, team_stats, how='left', left_on=['match_id', 'team_1_id'], right_on=['match_id', 'team_id'])
merged_df = pd.merge(merged_df, team_stats, how='left', left_on=['match_id', 'team_2_id'], right_on=['match_id', 'team_id'], suffixes=('_team_1', '_team_2'))

# Drop unnecessary columns and duplicates
merged_df.drop(
    [
        'adr', 
        'assists',
        'deaths',
        'fkdiff',
        'hs',
        'kdratio',
        'kills',
        'player_id',
        'team_id_x',
        'team_id_y',
        'team_id',
        'team_1_id',
        'team_2_id'
        ], 
    axis=1,
    inplace=True
    )

# Drop duplicates
merged_df.drop_duplicates(ignore_index=True, inplace=True)

# Order the dataframe by match ID
merged_df.sort_values(by='match_id')


,match_id,team_name,data_unix,map,winning_team,members_team_1,avg_adr_team_1,sum_assists_team_1,sum_deaths_team_1,sum_fkdiffs_team_1,...,mean_kdratio_team_1,sum_kills_team_1,members_team_2,avg_adr_team_2,sum_assists_team_2,sum_deaths_team_2,sum_fkdiffs_team_2,sum_hs_team_2,mean_kdratio_team_2,sum_kills_team_2
139535,12838,NIP,1347562800000,Inferno_se,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
139536,12838,PRiME,1347562800000,Inferno_se,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81401,12839,NIP,1347562800000,Mirage_ce,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81402,12839,PRiME,1347562800000,Mirage_ce,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72699,12840,Blight,1347562800000,Inferno_se,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71283,121302,LDLC,1622383200000,Mirage,1,"(15117, 8047, 13138, 147, 9656)",73.34,31.0,115.0,-1.0,...,0.7028,119.0,"(15370, 7499, 12782, 14932, 7511)",74.40,20.0,120.0,1.0,59.0,0.6570,114.0
7112,121304,Contact Light,1622394600000,Vertigo,2,"(14990, 19954, 15635, 17906, 20055)",74.32,24.0,132.0,2.0,...,0.6858,117.0,"(21025, 18156, 20127, 19575, 17087)",82.14,39.0,118.0,-3.0,71.0,0.7600,132.0
7111,121304,Trasko,1622394600000,Vertigo,2,"(14990, 19954, 15635, 17906, 20055)",74.32,24.0,132.0,2.0,...,0.6858,117.0,"(21025, 18156, 20127, 19575, 17087)",82.14,39.0,118.0,-3.0,71.0,0.7600,132.0
82804,121306,Case,1622397600000,Overpass,2,"(19686, 10570, 11213, 16954, 10557)",64.50,16.0,90.0,-4.0,...,0.5916,67.0,"(13602, 17956, 10096, 14419, 20090)",79.30,20.0,67.0,4.0,42.0,0.7584,89.0


***Drop irrelevant columns and reorder the final dataframe***

The final step will consist in rordering the columns and dropping the remain duplicates, nan values and irrelevant columns of the dataset.

In [610]:
# Drop irrelevant columns
total_data = merged_df
total_data.drop([
    'match_id',
    'data_unix'
], axis=1, inplace=True)

# Reorder the dataframe
order = [
    'members_team_1',
    'members_team_2',
    'map',  
    'avg_adr_team_1',
    'sum_assists_team_1', 
    'sum_deaths_team_1', 
    'sum_fkdiffs_team_1',
    'sum_hs_team_1', 
    'mean_kdratio_team_1', 
    'sum_kills_team_1',     
    'avg_adr_team_2', 
    'sum_assists_team_2',
    'sum_deaths_team_2', 
    'sum_fkdiffs_team_2', 
    'sum_hs_team_2',
    'mean_kdratio_team_2', 
    'sum_kills_team_2',
    'winning_team'
]
total_data = total_data.reindex(columns=order)

# remove columns with nan values
total_data.dropna(inplace=True)
# remove duplicate columns
total_data.drop_duplicates(ignore_index=True, inplace=True)

In [611]:
print(f'The final dataset has',total_data.shape[0], 'rows and',total_data.shape[1],'columns.')
total_data[:3]

The final dataset has 84040 rows and 18 columns.


,members_team_1,members_team_2,map,avg_adr_team_1,sum_assists_team_1,sum_deaths_team_1,sum_fkdiffs_team_1,sum_hs_team_1,mean_kdratio_team_1,sum_kills_team_1,avg_adr_team_2,sum_assists_team_2,sum_deaths_team_2,sum_fkdiffs_team_2,sum_hs_team_2,mean_kdratio_team_2,sum_kills_team_2,winning_team
0,"(168, 5737, 11247, 2492, 10814)","(5698, 10563, 2532, 5736, 7382)",Train,58.720000,4.0,89.0,-6.0,24.0,0.470000,48.0,92.000000,15.0,48.0,6.0,36.0,0.820000,89.0,2
1,"(483, 484, 2757, 7594, 3347)","(2469, 7398, 7592, 429, 4954)",Dust2,72.623042,21.0,54.0,12.0,32.0,0.692681,89.0,75.752211,15.0,90.0,-12.0,25.0,0.709822,54.0,1
2,"(10565, 11302, 10795, 10797, 10798)","(5698, 10563, 2532, 5736, 7382)",Cache,62.880000,11.0,96.0,-5.0,28.0,0.536000,66.0,81.040000,17.0,66.0,5.0,33.0,0.792000,96.0,2


***Save the dataset***

In [612]:
total_data.to_csv('../data/interim/clean_match_data.csv',index=False)

---

## Conclusions

1. The exploratory analysis of the dataset confirmed that there is a strong relationship between the player statistics in the game and the final outcome of the match.

2. The time variables have not provided strong correlation with the target variable, thus, they were eliminated from the dataset.

3. Player Null-values were replaced by their personal mean in order to keep their performance as equal as possible. Players with no records and null values were eliminated from the dataset.

4. In order to fulfill the requirements of the Machine Learning Model, it was decided to create a new dataset from the data given.

5. The new dataset includes the players ids and the average and sum of the individual statistics. The target is the winning team. A huge reordering was done in order to create a single row for every match that has all the needed information.

6. The clean dataset has 84040 rows and 18 columns and no null values. 